In [1]:
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip" !unzip -o -q ipopt-linux64
!unzip -o -q ipopt-linux64
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

import pyomo.environ as pyo

Modelo 1

In [13]:
import pyomo.environ as pyo
import pandas as pd

# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel("/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel("/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel("/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos
dicionario_media_dividendos = df_media_dividendos.mean().to_dict()

# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, within=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)
modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.20)

# Limites para variância dos dividend yields e dos preços dos ativos
limite_variancia_dy = 0.0016  # Exemplo de limite para a variância dos dividend yields
limite_variancia_dp = 0.0248  # Exemplo de limite para a variância dos preços dos ativos

# Restrições para variância dos dividend yields e dos preços dos ativos
modelo.restricao_variancia_dy = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dy)
modelo.restricao_variancia_dp = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dp)

# Função objetivo: Maximizar o retorno esperado da carteira
def funcao_objetivo_retorno(modelo):
    return sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos)

modelo.objetivo_retorno = pyo.Objective(rule=funcao_objetivo_retorno, sense=pyo.maximize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt')

# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    retorno_otimo = modelo.objetivo_retorno()
    variancia_dividendos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)
    variancia_precos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_otimo}')
    print(f'Variância total da Carteira Ótima: {variancia_dividendos_otima + variancia_precos_otima}')

else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

2 Modelo

In [14]:
# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel(r"/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel(r"/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel(r"/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos
dicionario_media_dividendos = df_media_dividendos.mean().to_dict()

# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Parâmetro de retorno mínimo esperado
retorno_minimo = 0.0552  # Valor de retorno mínimo fornecido na imagem

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, domain=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)

modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.20)

modelo.restricao_retorno = pyo.Constraint(rule=lambda modelo: sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos) >= retorno_minimo)

# Função objetivo: Minimizar a variância ponderada da carteira
def funcao_objetivo_risco(modelo):
    return sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) + \
           sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)

modelo.objetivo_risco = pyo.Objective(rule=funcao_objetivo_risco, sense=pyo.minimize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')


# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    risco_total = modelo.objetivo_risco()

    # Calculando o retorno esperado para a carteira ótima
    retorno_esperado = sum(modelo.pesos[ativo].value * dicionario_media_dividendos[ativo] for ativo in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_esperado}')
    print(f'Risco Total da Carteira (Variância): {risco_total}')
else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

Analisar os preços sombra das restrições ativa (preços sombra diferentes de zero)

In [78]:
import pyomo.environ as pyo
import pandas as pd

# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel("/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel("/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel("/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos
dicionario_media_dividendos = df_media_dividendos.mean().to_dict()

# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Adicionar suficiente dual para armazenar preços sombra
modelo.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, within=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)
modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.20)

# Limites para variância dos dividend yields e dos preços dos ativos
limite_variancia_dy = 0.0016  # Exemplo de limite para a variância dos dividend yields
limite_variancia_dp = 0.0248  # Exemplo de limite para a variância dos preços dos ativos

# Restrições para variância dos dividend yields e dos preços dos ativos
modelo.restricao_variancia_dy = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dy)
modelo.restricao_variancia_dp = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dp)

# Função objetivo: Maximizar o retorno esperado da carteira
def funcao_objetivo_retorno(modelo):
    return sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos)

modelo.objetivo_retorno = pyo.Objective(rule=funcao_objetivo_retorno, sense=pyo.maximize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')

# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    retorno_otimo = modelo.objetivo_retorno()
    variancia_dividendos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)
    variancia_precos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_otimo}')
    print(f'Variância total da Carteira Ótima: {variancia_dividendos_otima + variancia_precos_otima}')
    # Exibindo os preços sombra das restrições
    print("\nPreços Sombra das Restrições:")
    for c in modelo.component_objects(pyo.Constraint, active=True):
        for idx in c:
            try:
                # Verifica se a restrição está ativa (sem folga)
                if modelo.dual[c[idx]] != 0:
                    print(f"{c.name}['{idx}']:{modelo.dual[c[idx]]}")
            except KeyError:
                pass  # Ignora se o preço sombra não estiver disponível ou se a restrição não está ativa
else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

Cenário Positivo (Modelo 1) \\
Dividendos 10% maiores \\
Variância dos dividend yields 10% menor \\
Variância dos preços 20% menor \\
Concentração máxima de 25%

In [56]:
import pyomo.environ as pyo
import pandas as pd

# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel("/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel("/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel("/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos (aumentando em 10%)
dicionario_media_dividendos = {ativo: valor * 1.1 for ativo, valor in df_media_dividendos.mean().to_dict().items()}

# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())



# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, within=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)
modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.25)

# Limites para variância dos dividend yields e dos preços dos ativos
limite_variancia_dy = 0.0016 * 0.9 # Exemplo de limite para a variância dos dividend yields (caiu 10%)
limite_variancia_dp = 0.0248 * 0.8 # Exemplo de limite para a variância dos preços dos ativos (caiu 20%)

# Restrições para variância dos dividend yields e dos preços dos ativos
modelo.restricao_variancia_dy = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dy)
modelo.restricao_variancia_dp = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dp)

# Função objetivo: Maximizar o retorno esperado da carteira
def funcao_objetivo_retorno(modelo):
    return sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos)

modelo.objetivo_retorno = pyo.Objective(rule=funcao_objetivo_retorno, sense=pyo.maximize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')

# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    retorno_otimo = modelo.objetivo_retorno()
    variancia_dividendos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)
    variancia_precos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_otimo}')
    print(f'Variância total da Carteira Ótima: {variancia_dividendos_otima + variancia_precos_otima}')
    print(f'Variância dividendos da Carteira Ótima: {variancia_dividendos_otima }')
else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

Cenário Negativo (Modelo 1) \\
Dividendos 10% menores \\
Variância dos dividend yields 10% maior \\
Variância dos preços 20% maior \\
Concentração máxima de 15%

In [69]:
import pyomo.environ as pyo
import pandas as pd

# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel("/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel("/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel("/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos (caiu em 10%)
dicionario_media_dividendos = {ativo: valor * 0.9 for ativo, valor in df_media_dividendos.mean().to_dict().items()}


# Inicializar o modelo
modelo = pyo.ConcreteModel()


# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, within=pyo.NonNegativeReals)
# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)
modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.15) #(Concentração máxima de 15%)

# Limites para variância dos dividend yields e dos preços dos ativos
limite_variancia_dy = 0.0016 * 1.1 # Exemplo de limite para a variância dos dividend yields (aumentou 10%)
limite_variancia_dp = 0.0248 * 1.2 # Exemplo de limite para a variância dos preços dos ativos (aumentou 20%)

# Restrições para variância dos dividend yields e dos preços dos ativos
modelo.restricao_variancia_dy = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dy)
modelo.restricao_variancia_dp = pyo.Constraint(expr=sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) <= limite_variancia_dp)

# Função objetivo: Maximizar o retorno esperado da carteira
def funcao_objetivo_retorno(modelo):
    return sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos)

modelo.objetivo_retorno = pyo.Objective(rule=funcao_objetivo_retorno, sense=pyo.maximize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')

# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    retorno_otimo = modelo.objetivo_retorno()
    variancia_dividendos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)
    variancia_precos_otima = sum(modelo.pesos[i].value * modelo.pesos[j].value * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos)
    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_otimo}')
    print(f'Variância total da Carteira Ótima: {variancia_dividendos_otima + variancia_precos_otima}')
    print(f'Variância dividendos da Carteira Ótima: {variancia_dividendos_otima }')
else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

Cenário Positivo (Modelo 2) \\
Dividendos 10% maiores \\
Variância dos dividend yields 10% menor \\
Variância dos preços 20% menor \\
Concentração máxima de 25%

In [71]:
# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel(r"/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel(r"/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel(r"/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos (aumentou 10%)
dicionario_media_dividendos = {ativo: valor * 1.1 for ativo, valor in df_media_dividendos.mean().to_dict().items()}

# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Parâmetro de retorno mínimo esperado
retorno_minimo = 0.0552  # Valor de retorno mínimo fornecido na imagem

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, domain=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)

modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.25) #(Concentração máxima de 25%)

modelo.restricao_retorno = pyo.Constraint(rule=lambda modelo: sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos) >= retorno_minimo)

# Função objetivo: Minimizar a variância ponderada da carteira
def funcao_objetivo_risco(modelo):
    return sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) + \
           sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)

modelo.objetivo_risco = pyo.Objective(rule=funcao_objetivo_risco, sense=pyo.minimize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')


# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    risco_total = modelo.objetivo_risco()

    # Calculando o retorno esperado para a carteira ótima
    retorno_esperado = sum(modelo.pesos[ativo].value * dicionario_media_dividendos[ativo] for ativo in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_esperado}')
    print(f'Risco Total da Carteira (Variância): {risco_total}')
    print(f'Variância dividendos da Carteira Ótima: {variancia_dividendos_otima }')

else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

Cenário Negativo (Modelo 2) \\
Dividendos 10% menores \\
Variância dos dividend yields 10% maior \\
Variância dos preços 20% maior \\
Concentração máxima de 15%

In [70]:
# Carregar os arquivos Excel para DataFrames do Pandas
df_variancia_precos = pd.read_excel(r"/content/Preços - Variância e Covariância.xlsx")
df_variancia_dividendos = pd.read_excel(r"/content/Dividendos - Variância e Covariância.xlsx")
df_media_dividendos = pd.read_excel(r"/content/Dividendos - Média - 5 Anos.xlsx")

# Extrair nomes dos ativos (ignorando a primeira coluna que é o nome do index)
nomes_ativos = df_variancia_precos.columns[1:]

# Criar um dicionário para as médias dos dividendos (caiu 10%)
dicionario_media_dividendos = {ativo: valor * 0.9 for ativo, valor in df_media_dividendos.mean().to_dict().items()}


# Inicializar o modelo
modelo = pyo.ConcreteModel()

# Definir conjuntos
modelo.ativos = pyo.Set(initialize=nomes_ativos)

# Definir parâmetros
modelo.media_dividendos = pyo.Param(modelo.ativos, initialize=dicionario_media_dividendos)
modelo.variancia_precos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_precos.set_index(df_variancia_precos.columns[0]).stack().to_dict())
modelo.variancia_dividendos = pyo.Param(modelo.ativos, modelo.ativos, initialize=df_variancia_dividendos.set_index(df_variancia_dividendos.columns[0]).stack().to_dict())

# Parâmetro de retorno mínimo esperado
retorno_minimo = 0.0552  # Valor de retorno mínimo fornecido na imagem

# Variáveis de decisão: pesos dos ativos na carteira
modelo.pesos = pyo.Var(modelo.ativos, domain=pyo.NonNegativeReals)

# Restrições
modelo.soma_pesos = pyo.Constraint(expr=sum(modelo.pesos[i] for i in modelo.ativos) == 1)

modelo.concentracao_maxima = pyo.Constraint(modelo.ativos, rule=lambda modelo, i: modelo.pesos[i] <= 0.15) #(Concentração máxima de 15%)

modelo.restricao_retorno = pyo.Constraint(rule=lambda modelo: sum(modelo.pesos[i] * modelo.media_dividendos[i] for i in modelo.ativos) >= retorno_minimo)

# Função objetivo: Minimizar a variância ponderada da carteira
def funcao_objetivo_risco(modelo):
    return sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_precos[i,j] for i in modelo.ativos for j in modelo.ativos) + \
           sum(modelo.pesos[i] * modelo.pesos[j] * modelo.variancia_dividendos[i,j] for i in modelo.ativos for j in modelo.ativos)

modelo.objetivo_risco = pyo.Objective(rule=funcao_objetivo_risco, sense=pyo.minimize)

# Configurar o solver
solver = pyo.SolverFactory('ipopt', executable='/content/ipopt')


# Resolvendo o modelo
resultado = solver.solve(modelo, tee=True)

# Checando se a solução foi encontrada
if (resultado.solver.status == pyo.SolverStatus.ok) and (resultado.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Criando um dicionário para armazenar os resultados
    pesos_otimos = {ativo: modelo.pesos[ativo].value for ativo in modelo.ativos}
    risco_total = modelo.objetivo_risco()

    # Calculando o retorno esperado para a carteira ótima
    retorno_esperado = sum(modelo.pesos[ativo].value * dicionario_media_dividendos[ativo] for ativo in modelo.ativos)

    print(f'Pesos da Carteira Ótima: {pesos_otimos}')
    print(f'Retorno Esperado da Carteira Ótima: {retorno_esperado}')
    print(f'Risco Total da Carteira (Variância): {risco_total}')
    print(f'Variância dividendos da Carteira Ótima: {variancia_dividendos_otima }')
else:
    print("A solução ótima não foi encontrada.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:       20
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot